# Assignment 3 — Queries


In [ ]:
from DbConnector import DbConnector
from statistics import median
import pandas as pd

conn = DbConnector()
db = conn.db
print("Connected to:", db.name)

def show_table(rows, limit=15):
    df = pd.DataFrame(rows)
    if len(df) > limit:
        display(df.head(limit))
        print(f"... {len(df) - limit} more rows not shown ...")
    else:
        display(df)
    return df


## Q1 — Directors with 5 or more movies

This query looks at directors who have made at least five films and compares their box office performance and ratings.  
The idea is to identify who consistently directs successful movies both commercially and critically.  
It takes the crew data from `credits`, filters only those entries where the job is “Director”, and then joins them with the `movies` collection to get the revenue and average vote.  
Each director is grouped with all their movies, the number of films is counted, and the average TMDB rating is calculated.  
Finally, the median revenue is computed in Python since it is easier and avoids extra aggregation stages.


In [ ]:
pipeline_q1 = [
    {"$unwind": "$crew"},
    {"$match": {"crew.job": "Director"}},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$group": {
        "_id": "$crew.name",
        "movie_count": {"$sum": 1},
        "revenues": {"$push": "$mv.revenue"},
        "avg_vote": {"$avg": "$mv.vote_average"}
    }},
    {"$match": {"movie_count": {"$gte": 5}}},
    {"$project": {
        "_id": 0,
        "director": "$_id",
        "movie_count": 1,
        "avg_vote": {"$round": ["$avg_vote", 2]},
        "revenues": 1
    }}
]

rows_q1 = list(db.credits.aggregate(pipeline_q1, allowDiskUse=True))
for r in rows_q1:
    valid = [x for x in (r.get("revenues") or []) if isinstance(x, (int, float))]
    r["median_revenue"] = float(median(valid)) if valid else None
    r.pop("revenues", None)

rows_q1_sorted = sorted(rows_q1, key=lambda x: (x["median_revenue"] or -1), reverse=True)[:20]
show_table(rows_q1_sorted)


## Q2 — Actor pairs with at least three co-appearances

This query finds actor pairs that frequently appear together in movies.  
It unfolds the cast list twice for the same film to create all possible pairs and keeps only one order of each pair to avoid duplicates.  
The results are joined with `movies` to retrieve the rating of each film, and then the pairs are grouped by the two actor names.  
For every pair, the number of shared movies and their average rating are calculated, keeping only those who have worked together three or more times.


In [ ]:
pipeline_q2 = [
    {"$project": {"movie_id": 1, "cast": 1}},
    {"$unwind": "$cast"},
    {"$project": {"movie_id": 1, "actor1": "$cast"}},
    {"$lookup": {
        "from": "credits",
        "localField": "movie_id",
        "foreignField": "movie_id",
        "as": "same"
    }},
    {"$unwind": "$same"},
    {"$unwind": "$same.cast"},
    {"$project": {
        "movie_id": 1,
        "a1_id": "$actor1.id",
        "a1_name": "$actor1.name",
        "a2_id": "$same.cast.id",
        "a2_name": "$same.cast.name"
    }},
    {"$match": {"$expr": {"$gt": ["$a2_id", "$a1_id"]}}},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$group": {
        "_id": {"a1": "$a1_name", "a2": "$a2_name"},
        "co_appearances": {"$sum": 1},
        "avg_vote": {"$avg": "$mv.vote_average"}
    }},
    {"$match": {"co_appearances": {"$gte": 3}}},
    {"$project": {
        "_id": 0,
        "actor_1": "$_id.a1",
        "actor_2": "$_id.a2",
        "co_appearances": 1,
        "avg_vote": {"$round": ["$avg_vote", 2]}
    }},
    {"$sort": {"co_appearances": -1, "avg_vote": -1}},
    {"$limit": 25}
]

rows_q2 = list(db.credits.aggregate(pipeline_q2, allowDiskUse=True))
show_table(rows_q2)


## Q3 — Actors with the widest genre variety

The goal of this query is to highlight actors who have worked across the most diverse set of genres.  
After connecting each cast member with the corresponding movie information, the list of genres for each movie is expanded.  
The query then groups by actor name and collects a set of distinct genre names to count how many different ones appear.  
Only actors with at least ten movies are included, and the final list is sorted by the number of distinct genres.


In [ ]:
pipeline_q3 = [
    {"$unwind": "$cast"},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$unwind": "$mv.genres"},
    {"$group": {
        "_id": "$cast.name",
        "movie_count": {"$sum": 1},
        "genres": {"$addToSet": "$mv.genres.name"}
    }},
    {"$match": {"movie_count": {"$gte": 10}}},
    {"$project": {
        "_id": 0,
        "actor": "$_id",
        "movie_count": 1,
        "genre_count": {"$size": "$genres"},
        "sample_genres": {"$slice": ["$genres", 5]}
    }},
    {"$sort": {"genre_count": -1, "movie_count": -1}},
    {"$limit": 25}
]

rows_q3 = list(db.credits.aggregate(pipeline_q3, allowDiskUse=True))
show_table(rows_q3)


## Q4 — Top movie collections by total revenue

This query focuses on film collections or sagas and measures their financial success.  
It selects movies that belong to a collection, groups them by the collection name, and sums their total revenue.  
The aggregation also counts how many films belong to each collection and gathers all their ratings.  
The median of these ratings is calculated afterwards in Python, and the final result displays the ten most profitable collections.


In [ ]:
pipeline_q4 = [
    {"$match": {"belongs_to_collection": {"$type": "object"}}},
    {"$group": {
        "_id": "$belongs_to_collection.name",
        "movie_count": {"$sum": 1},
        "total_revenue": {"$sum": "$revenue"},
        "ratings": {"$push": "$vote_average"},
        "first_date": {"$min": "$release_date"},
        "last_date": {"$max": "$release_date"}
    }},
    {"$match": {"movie_count": {"$gte": 3}}},
    {"$sort": {"total_revenue": -1}},
    {"$limit": 10}
]

rows_q4 = list(db.movies.aggregate(pipeline_q4, allowDiskUse=True))
for r in rows_q4:
    valid = [x for x in (r.get("ratings") or []) if isinstance(x, (int, float))]
    r["median_vote"] = float(median(valid)) if valid else None
    r.pop("ratings", None)

show_table(rows_q4)


## Q5 — Median runtime per decade and primary genre

This query explores how movie runtimes vary by decade and genre.  
The release year of each film is converted into a decade label such as “1980s” or “2000s”.  
The first genre listed in the movie data is treated as the primary genre for grouping purposes.  
Movies are grouped by both decade and genre, and for each group the total number of films is recorded.  
The median runtime is then computed in Python to give a clean summary of the central tendency for each category.


In [ ]:
pipeline_q5 = [
    {"$addFields": {
        "year": {"$year": "$release_date"},
        "primary_genre": {
            "$cond": [
                {"$gt": [{"$size": {"$ifNull": ["$genres", []]}}, 0]},
                {"$arrayElemAt": ["$genres.name", 0]},
                None
            ]
        }
    }},
    {"$addFields": {
        "decade": {"$concat": [
            {"$toString": {"$multiply": [{"$floor": {"$divide": ["$year", 10]}}, 10]}},
            "s"
        ]}
    }},
    {"$group": {
        "_id": {"decade": "$decade", "genre": "$primary_genre"},
        "runtimes": {"$push": "$runtime"},
        "movie_count": {"$sum": 1}
    }}
]

rows_q5 = list(db.movies.aggregate(pipeline_q5, allowDiskUse=True))
for r in rows_q5:
    vals = [x for x in (r.get("runtimes") or []) if isinstance(x, (int, float))]
    r["median_runtime"] = float(median(vals)) if vals else None
    r.pop("runtimes", None)

rows_q5_sorted = sorted(rows_q5, key=lambda x: (x["_id"]["decade"], -(x["median_runtime"] or 0)))[:40]
show_table(rows_q5_sorted)


## Q6 — Share of women in top-5 billed cast by decade

This query measures how the presence of women among the top-billed actors has changed over time.  
It keeps only cast entries where the order is less than five, which corresponds to the top of the cast list.  
For each movie, it counts how many of these actors are women using the `gender` value equal to 1.  
The ratio of female actors within the top five is then averaged by decade based on each movie’s release date.


In [ ]:
pipeline_q6 = [
    {"$project": {"movie_id": 1, "cast": 1}},
    {"$unwind": "$cast"},
    {"$match": {"cast.order": {"$lt": 5}}},
    {"$group": {
        "_id": "$movie_id",
        "top5_total": {"$sum": 1},
        "top5_females": {"$sum": {"$cond": [{"$eq": ["$cast.gender", 1]}, 1, 0]}}
    }},
    {"$project": {
        "_id": 0,
        "movie_id": "$_id",
        "ratio": {
            "$cond": [{"$gt": ["$top5_total", 0]}, {"$divide": ["$top5_females", "$top5_total"]}, None]
        }
    }},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$addFields": {"year": {"$year": "$mv.release_date"}}},
    {"$addFields": {
        "decade": {"$concat": [
            {"$toString": {"$multiply": [{"$floor": {"$divide": ["$year", 10]}}, 10]}},
            "s"
        ]}
    }},
    {"$group": {"_id": "$decade", "avg_ratio": {"$avg": "$ratio"}, "movies": {"$sum": 1}}},
    {"$project": {"_id": 0, "decade": "$_id", "avg_ratio": {"$round": ["$avg_ratio", 3]}, "movies": 1}},
    {"$sort": {"decade": 1}}
]

rows_q6 = list(db.credits.aggregate(pipeline_q6, allowDiskUse=True))
show_table(rows_q6, limit=50)


## Q7 — Noir and neo-noir movies (regex search)

The goal of this query is to find movies described as noir or neo-noir in their overview or tagline.  
Since there is no text index in the database, regular expressions are used to match both words in a case-insensitive way.  
To avoid irrelevant titles, only movies with at least fifty votes are included.  
The output lists the top results sorted by rating.

In [ ]:
pipeline_q7 = [
    {"$match": {
        "$or": [
            {"overview": {"$regex": "\\bneo-?noir\\b", "$options": "i"}},
            {"overview": {"$regex": "\\bnoir\\b", "$options": "i"}},
            {"tagline":  {"$regex": "\\bneo-?noir\\b", "$options": "i"}},
            {"tagline":  {"$regex": "\\bnoir\\b", "$options": "i"}}
        ],
        "vote_count": {"$gte": 50}
    }},
    {"$project": {"_id": 0, "title": 1, "release_date": 1, "vote_average": 1, "vote_count": 1}},
    {"$sort": {"vote_average": -1, "vote_count": -1}},
    {"$limit": 25}
]

rows_q7 = list(db.movies.aggregate(pipeline_q7, allowDiskUse=True))
show_table(rows_q7, limit=25)


## Q8 — Best director–actor collaborations

This query highlights pairs of directors and actors who have frequently worked together and achieved strong average ratings.  
It first extracts each movie’s director, joins the same movie’s cast to create all possible director–actor pairs,  
and then connects to the `movies` collection to include vote counts.  
Only movies with at least one hundred votes are considered to keep the sample meaningful,  
and only pairs with at least three collaborations are kept in the final ranking.


In [ ]:
pipeline_q8 = [
    {"$unwind": "$crew"},
    {"$match": {"crew.job": "Director"}},
    {"$project": {"movie_id": 1, "director": "$crew.name"}},
    {"$lookup": {
        "from": "credits",
        "localField": "movie_id",
        "foreignField": "movie_id",
        "as": "cr"
    }},
    {"$unwind": "$cr"},
    {"$unwind": "$cr.cast"},
    {"$project": {"movie_id": 1, "director": 1, "actor": "$cr.cast.name"}},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$match": {"mv.vote_count": {"$gte": 100}}},
    {"$group": {
        "_id": {"director": "$director", "actor": "$actor"},
        "collabs": {"$sum": 1},
        "avg_vote": {"$avg": "$mv.vote_average"}
    }},
    {"$match": {"collabs": {"$gte": 3}}},
    {"$project": {
        "_id": 0,
        "director": "$_id.director",
        "actor": "$_id.actor",
        "collabs": 1,
        "avg_vote": {"$round": ["$avg_vote", 2]}
    }},
    {"$sort": {"avg_vote": -1, "collabs": -1}},
    {"$limit": 25}
]

rows_q8 = list(db.credits.aggregate(pipeline_q8, allowDiskUse=True))
show_table(rows_q8, limit=25)


## Q9 — Non-English languages connected to the United States

This query checks which non-English original languages appear most often in movies that have a connection to the United States.  
A connection is counted if the movie lists the United States either as a production country or as the origin of one of its production companies.  
The aggregation counts how many movies match each non-English language and returns the ten most common ones.


In [ ]:
pipeline_q9 = [
    {"$match": {"original_language": {"$ne": "en"}}},
    {"$addFields": {
        "has_us_country": {"$gt": [{"$size": {
            "$filter": {
                "input": {"$ifNull": ["$production_countries", []]},
                "as": "c",
                "cond": {"$eq": ["$$c.iso_3166_1", "US"]}
            }}}, 0]},
        "has_us_company": {"$gt": [{"$size": {
            "$filter": {
                "input": {"$ifNull": ["$production_companies", []]},
                "as": "p",
                "cond": {"$eq": ["$$p.origin_country", "US"]}
            }}}, 0]}
    }},
    {"$match": {"$or": [{"has_us_country": True}, {"has_us_company": True}]}},
    {"$group": {"_id": "$original_language", "count": {"$sum": 1}}},
    {"$project": {"_id": 0, "language": "$_id", "count": 1}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

rows_q9 = list(db.movies.aggregate(pipeline_q9, allowDiskUse=True))
show_table(rows_q9, limit=10)


## Q10 — Per-user statistics: number of ratings, variance, and genre diversity

The last query calculates a few basic statistics for each user.  
It joins `ratings` with `links` and then `movies` to bring the genre information of every rated movie.  
Each user’s set of distinct genres is collected to count how varied their watched content is.  
The population standard deviation of the ratings is used to compute the variance.  
To keep the results stable, only users with at least twenty ratings are included.


In [ ]:
pipeline_q10 = [
    {"$lookup": {"from": "links", "localField": "movieId", "foreignField": "movieId", "as": "lk"}},
    {"$unwind": "$lk"},
    {"$lookup": {"from": "movies", "localField": "lk.tmdbId", "foreignField": "_id", "as": "mv"}},
    {"$unwind": "$mv"},
    {"$unwind": {"path": "$mv.genres", "preserveNullAndEmptyArrays": True}},
    {"$group": {
        "_id": "$userId",
        "ratings_count": {"$sum": 1},
        "genres": {"$addToSet": "$mv.genres.name"},
        "std_pop": {"$stdDevPop": "$rating"}
    }},
    {"$project": {
        "_id": 0,
        "userId": "$_id",
        "ratings_count": 1,
        "genre_count": {"$size": {"$setDifference": ["$genres", [None]]}},
        "variance": {
            "$cond": [{"$ne": ["$std_pop", None]}, {"$multiply": ["$std_pop", "$std_pop"]}, None]
        }
    }},
    {"$match": {"ratings_count": {"$gte": 20}}},
    {"$sort": {"genre_count": -1, "variance": -1, "ratings_count": -1}},
    {"$limit": 30}
]

rows_q10 = list(db.ratings.aggregate(pipeline_q10, allowDiskUse=True))
show_table(rows_q10, limit=30)
